## EDA

## Inicialización

In [196]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from scipy import stats as scipy_stats

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.max_rows', 100)  # Mostrar hasta 100 filas

## Carga de datos

In [197]:
# import data
df = pd.read_parquet('../data/raw/yellow_tripdata_2022-05.parquet', engine='pyarrow')

# Taxi Zone Lookup Table
df_zones = pd.read_excel('../data/raw/taxi_zone_lookup.xlsx')

In [198]:
# Crear función para cambiar nombres de columnas
def to_snake_case(name):
    name = name.replace(' ', '_')
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

# Rename the columns using the to_snake_case function
df.columns = [to_snake_case(col) for col in df.columns]
df_zones.columns = [to_snake_case(col) for col in df_zones.columns]

print(df.columns)
print(df_zones.columns)

Index(['vendor_id', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'ratecode_id', 'store_and_fwd_flag',
       'pu_location_id', 'do_location_id', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'congestion_surcharge',
       'airport_fee'],
      dtype='object')
Index(['location_id', 'borough', 'zone', 'service_zone'], dtype='object')


In [199]:
rows_before = len(df)
rows_before

3588295

In [200]:
# Mapeo de códigos

rate_code_mapping = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negocios",
    6: "Group ride",
    99: "Null/unknown"
    }

vendor_mapping = {
    1: "Creative Mobile Technologies, LLC",
    2: "Curb Mobility, LLC",
    6: "Myle Technologies Inc",
    7: "Heli Taxi"
    }

payment_mapping = {
    0: "Flex Fare trip",
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
    }

# Mapeo de store_and_fwd_flag
store_and_fwd_mapping = {
    "Y": "Store and forward",
    "N": "Not a store and forward trip"
    }


## Analisis exploratorio

In [201]:
# Print data
df.sample(10)

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
620463,2,2022-05-06 16:34:47,2022-05-06 16:55:43,1.00,2.33,1.00,N,161,140,1,14.50,1.00,0.50,3.00,0.00,0.30,21.80,2.50,0.00
2838632,2,2022-05-25 20:44:04,2022-05-25 20:52:19,1.00,1.48,1.00,N,236,239,1,7.50,0.50,0.50,2.26,0.00,0.30,13.56,2.50,0.00
2525939,2,2022-05-23 07:35:05,2022-05-23 07:50:47,1.00,2.35,1.00,N,142,140,1,12.00,0.00,0.50,2.30,0.00,0.30,17.60,2.50,0.00
2863567,2,2022-05-26 02:38:31,2022-05-26 03:00:52,3.00,4.62,1.00,N,164,87,1,18.50,0.50,0.50,2.00,0.00,0.30,24.30,2.50,0.00
190115,1,2022-05-03 05:56:49,2022-05-03 06:07:15,1.00,2.50,1.00,N,186,229,1,10.50,3.00,0.50,2.85,0.00,0.30,17.15,2.50,0.00
2926875,2,2022-05-26 16:59:57,2022-05-26 17:20:04,1.00,2.30,1.00,N,161,75,1,13.50,1.00,0.50,2.67,0.00,0.30,20.47,2.50,0.00
558298,2,2022-05-06 02:22:21,2022-05-06 02:29:58,1.00,2.45,1.00,N,43,74,1,9.00,0.50,0.50,2.56,0.00,0.30,15.36,2.50,0.00
2785296,1,2022-05-25 13:38:34,2022-05-25 14:03:22,1.00,3.20,1.00,N,141,238,1,17.00,2.50,0.50,4.05,0.00,0.30,24.35,2.50,0.00
18282,1,2022-05-01 07:47:36,2022-05-01 08:03:11,1.00,0.00,1.00,N,153,242,1,24.20,0.00,0.50,0.00,0.00,0.30,25.00,0.00,0.00
490547,2,2022-05-05 16:47:25,2022-05-05 16:52:46,1.00,0.89,1.00,N,141,140,1,5.50,1.00,0.50,2.94,0.00,0.30,12.74,2.50,0.00


In [202]:
# Show information of the data
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3588295 entries, 0 to 3588294
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   vendor_id              3588295 non-null  int64         
 1   tpep_pickup_datetime   3588295 non-null  datetime64[us]
 2   tpep_dropoff_datetime  3588295 non-null  datetime64[us]
 3   passenger_count        3458771 non-null  float64       
 4   trip_distance          3588295 non-null  float64       
 5   ratecode_id            3458771 non-null  float64       
 6   store_and_fwd_flag     3458771 non-null  object        
 7   pu_location_id         3588295 non-null  int64         
 8   do_location_id         3588295 non-null  int64         
 9   payment_type           3588295 non-null  int64         
 10  fare_amount            3588295 non-null  float64       
 11  extra                  3588295 non-null  float64       
 12  mta_tax                35882

In [203]:
# Show descriptive statistics
df.describe()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,"3,588,295.00",3588295,3588295,"3,458,771.00","3,588,295.00","3,458,771.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,588,295.00","3,458,771.00","3,458,771.00"
mean,1.71,2022-05-16 07:50:29.219312,2022-05-16 08:08:42.275203,1.39,6.86,1.37,164.57,162.55,1.18,15.17,1.02,0.49,2.82,0.58,0.30,22.08,2.28,0.10
min,1.00,2003-01-01 00:06:06,2003-01-01 00:31:38,0.00,0.00,1.00,1.00,1.00,0.00,"-1,311.50",-4.50,-0.50,-145.70,-50.75,-0.30,"-1,314.80",-2.50,-1.25
25%,1.00,2022-05-08 18:14:16.500000,2022-05-08 18:32:36,1.00,1.15,1.00,132.00,113.00,1.00,7.00,0.00,0.50,1.00,0.00,0.30,12.35,2.50,0.00
50%,2.00,2022-05-16 09:14:42,2022-05-16 09:33:15,1.00,1.96,1.00,162.00,162.00,1.00,10.50,0.50,0.50,2.16,0.00,0.30,16.30,2.50,0.00
75%,2.00,2022-05-23 18:03:17,2022-05-23 18:21:03.500000,1.00,3.73,1.00,234.00,234.00,1.00,17.00,2.50,0.50,3.46,0.00,0.30,23.76,2.50,0.00
max,6.00,2022-06-01 23:55:30,2022-06-02 00:03:51,9.00,"357,192.65",99.00,265.00,265.00,4.00,"6,966.50",8.80,3.30,665.00,813.75,0.30,"6,970.80",2.75,1.25
std,0.49,NaN,NaN,0.96,690.85,5.24,65.63,70.28,0.51,14.89,1.26,0.09,3.37,2.17,0.05,18.49,0.75,0.34


In [204]:
# Verificar si hay datos duplicados
df.duplicated().sum()

0

In [205]:
# Resumen compacto de todas las columnas en una tabla
summary = pd.DataFrame({
    'Columna': df.columns,
    'Únicos': [df[col].nunique() for col in df.columns],
    'Nulos': [df[col].isna().sum() for col in df.columns],
    'Pct_Nulos': [round(df[col].isna().sum() / len(df) * 100, 2) for col in df.columns],
    'Top_Valor': [df[col].value_counts().index[0] if len(df[col]) > 0 else None for col in df.columns],
    'Top_Freq': [df[col].value_counts().values[0] if len(df[col]) > 0 else 0 for col in df.columns]
})

display(summary)

,Columna,Únicos,Nulos,Pct_Nulos,Top_Valor,Top_Freq
0,vendor_id,4,0,0.00,2,2527997
1,tpep_pickup_datetime,1754711,0,0.00,2022-05-06 19:22:00,16
2,tpep_dropoff_datetime,1752869,0,0.00,2022-05-30 00:00:00,34
3,passenger_count,10,129524,3.61,1.00,2549880
4,trip_distance,4771,0,0.00,1.00,49609
5,ratecode_id,7,129524,3.61,1.00,3256985
6,store_and_fwd_flag,2,129524,3.61,N,3404113
7,pu_location_id,259,0,0.00,132,175943
8,do_location_id,261,0,0.00,236,155644
9,payment_type,5,0,0.00,1,2720127


In [206]:
# Porcentaje de nulos (total no por columnas)
print(df.isna().sum() / len(df) * 100)

vendor_id               0.00
tpep_pickup_datetime    0.00
tpep_dropoff_datetime   0.00
passenger_count         3.61
trip_distance           0.00
ratecode_id             3.61
store_and_fwd_flag      3.61
pu_location_id          0.00
do_location_id          0.00
payment_type            0.00
fare_amount             0.00
extra                   0.00
mta_tax                 0.00
tip_amount              0.00
tolls_amount            0.00
improvement_surcharge   0.00
total_amount            0.00
congestion_surcharge    3.61
airport_fee             3.61
dtype: float64


In [207]:
# Filas con nulos
df[df.isna().any(axis=1)]

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
3458771,2,2022-05-01 00:01:17,2022-05-01 00:18:38,NaN,4.93,NaN,None,166,163,0,23.21,0.00,0.50,2.94,0.00,0.30,29.45,NaN,NaN
3458772,2,2022-05-01 00:40:00,2022-05-01 00:50:00,NaN,1.48,NaN,None,190,97,0,10.00,0.00,0.50,2.38,0.00,0.30,13.18,NaN,NaN
3458773,1,2022-05-01 00:08:38,2022-05-01 00:28:26,NaN,3.90,NaN,None,239,234,0,16.50,0.50,0.50,4.06,0.00,0.30,24.36,NaN,NaN
3458774,2,2022-05-01 00:34:59,2022-05-01 01:12:00,NaN,13.16,NaN,None,61,151,0,43.79,0.00,0.50,10.53,0.00,0.30,57.62,NaN,NaN
3458775,2,2022-05-01 00:12:00,2022-05-01 00:25:00,NaN,4.29,NaN,None,79,262,0,17.65,0.00,0.50,3.00,0.00,0.30,23.95,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588290,2,2022-05-31 23:40:19,2022-06-01 00:01:20,NaN,0.00,NaN,None,36,181,0,20.97,0.00,0.50,1.00,0.00,0.30,22.77,NaN,NaN
3588291,2,2022-05-31 23:52:12,2022-06-01 00:06:40,NaN,2.91,NaN,None,144,256,0,11.68,0.00,0.00,2.51,0.00,0.30,16.99,NaN,NaN
3588292,2,2022-05-31 23:27:00,2022-05-31 23:48:00,NaN,3.96,NaN,None,161,145,0,14.54,0.00,0.50,0.00,0.00,0.30,17.84,NaN,NaN
3588293,2,2022-05-31 23:34:12,2022-06-01 00:00:13,NaN,6.27,NaN,None,145,17,0,24.01,0.00,0.50,5.54,0.00,0.30,30.35,NaN,NaN


In [208]:
# Verificar códigos únicos en las columnas de pickup y dropoff tiene zonas fuera de New York
out_ny = df[df['pu_location_id'].isin([264, 265]) | df['do_location_id'].isin([264, 265])]
print(f"Total de viajes fuera de New York: {len(out_ny)}")
print(f"Porcentaje de viajes fuera de New York: {len(out_ny)/len(df)*100:.2f}%")


Total de viajes fuera de New York: 65029
Porcentaje de viajes fuera de New York: 1.81%


### Conclusión y siguientes pasos

Hemos encontrado en nuestro anlisis exploratorio que hay variables con nulos y valores negativos en el dataset. Por otro lado, hay variables que no forman parte ni tienen realción con nuestros target, estos serán eliminados.

- Las variables 'extra', 'mta_tax', 'improvement_surcharge', 'tolls_amount', 'congestion_surcharge', 'store_and_fwd_flag', 'total_amount', 'tip_amount', 'airport_fee' no tienen relación con el costo del viaje, por lo que se eliminarán.
- Los nulos representan el 3.61% de los datos. 
- Un venddor con ID no reconocido, este se procederá a eliminar ya que son solo 14 registros.
- Hay códigos no validos para las zonas de pickup y dropoff, estos se eliminarán ya que no pertenecen a la zona de estudio, New York.
- Hay valores negativos para el fare amount que deben ser analizados.


## Preprocesamiento de datos

Primero eliminaremos las columnas mencionadas anteriormente.

In [209]:
df = df.drop(columns=[
    'extra',
    'mta_tax',
    'improvement_surcharge',
    'tolls_amount',
    'congestion_surcharge',
    'store_and_fwd_flag',
    'total_amount',
    'tip_amount',
    'airport_fee'
    ]
             )
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3588295 entries, 0 to 3588294
Data columns (total 10 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   vendor_id              3588295 non-null  int64         
 1   tpep_pickup_datetime   3588295 non-null  datetime64[us]
 2   tpep_dropoff_datetime  3588295 non-null  datetime64[us]
 3   passenger_count        3458771 non-null  float64       
 4   trip_distance          3588295 non-null  float64       
 5   ratecode_id            3458771 non-null  float64       
 6   pu_location_id         3588295 non-null  int64         
 7   do_location_id         3588295 non-null  int64         
 8   payment_type           3588295 non-null  int64         
 9   fare_amount            3588295 non-null  float64       
dtypes: datetime64[us](2), float64(4), int64(4)
memory usage: 273.8 MB


Vamos a filtrar los registrs con vendor id igual a 5.

In [210]:
# Filtrar registros del vendor 5
df = df[df['vendor_id'] != 5]

# Verificar resultados
df['vendor_id'].value_counts()

vendor_id
2    2527997
1    1054130
6       6154
Name: count, dtype: int64

También en lo códigos de pickup y dropoff hay códigos que no pertencen a la zona de estudio, New York, estos se eliminarán.

In [211]:
# Filtrar registros fuera de New York
df = df[
    ~df['pu_location_id'].isin([264, 265]) & 
    ~df['do_location_id'].isin([264, 265])
]

# Ver que se filtraron en las columnas de pickup y dropoff
print(df['pu_location_id'].isin([264, 265]).any())
print(df['do_location_id'].isin([264, 265]).any())

False
False


In [212]:
df.isnull().sum() / len(df) * 100

vendor_id               0.00
tpep_pickup_datetime    0.00
tpep_dropoff_datetime   0.00
passenger_count         3.47
trip_distance           0.00
ratecode_id             3.47
pu_location_id          0.00
do_location_id          0.00
payment_type            0.00
fare_amount             0.00
dtype: float64

Ahora, haremos el mapeo de códdigos de las columas vendor_id, ratecode_id, payment_type, pu_location_id y do_location_id para entender un poco mejor las categorías de los datos. Además, añadiremos una columna que indique el tiempo de viaje en minutos.

In [213]:
# Mapeo de códigos

rate_code_mapping = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negocios",
    6: "Group ride",
    99: "Null/unknown"
    }

vendor_mapping = {
    1: "Creative Mobile Technologies, LLC",
    2: "Curb Mobility, LLC",
    6: "Myle Technologies Inc",
    7: "Heli Taxi"
    }

payment_mapping = {
    0: "Flex Fare trip",
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
    }

In [214]:
# Mapeo de columnas

# Mapear las localizaciones de los viajes con los nombres de las distritos, zonas y zonas de servicio.
df['pu_location_zone'] = df['pu_location_id'].map(df_zones.set_index('location_id')['zone'])
df['do_location_zone'] = df['do_location_id'].map(df_zones.set_index('location_id')['zone'])

df['do_location_district'] = df['do_location_id'].map(df_zones.set_index('location_id')['borough'])
df['pu_location_district'] = df['pu_location_id'].map(df_zones.set_index('location_id')['borough'])

df['do_location_service_zone'] = df['do_location_id'].map(df_zones.set_index('location_id')['service_zone'])
df['pu_location_service_zone'] = df['pu_location_id'].map(df_zones.set_index('location_id')['service_zone'])

# Verificar resultados
df[['pu_location_id', 'pu_location_zone', 'pu_location_district', 'pu_location_service_zone', 'do_location_id', 'do_location_zone', 'do_location_district', 'do_location_service_zone']].head()

# Mapear las columnas de ratecode_id, vendor_id y payment_type
df['rate_code'] = df['ratecode_id'].map(rate_code_mapping)
df['vendor'] = df['vendor_id'].map(vendor_mapping)
df['payment_type'] = df['payment_type'].map(payment_mapping)

# Verificar resultados
df.head()


,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,payment_type,fare_amount,pu_location_zone,do_location_zone,do_location_district,pu_location_district,do_location_service_zone,pu_location_service_zone,rate_code,vendor
0,1,2022-05-01 00:00:36,2022-05-01 00:19:18,1.00,4.10,1.00,246,151,Cash,17.00,West Chelsea/Hudson Yards,Manhattan Valley,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
1,1,2022-05-01 00:27:44,2022-05-01 00:41:33,1.00,2.30,1.00,238,74,Cash,11.00,Upper West Side North,East Harlem North,Manhattan,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
2,1,2022-05-01 00:59:00,2022-05-01 01:14:22,1.00,4.20,1.00,163,260,Cash,15.50,Midtown North,Woodside,Queens,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
3,1,2022-05-01 00:48:18,2022-05-01 01:28:02,1.00,0.00,1.00,79,182,Credit card,41.20,East Village,Parkchester,Bronx,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
4,1,2022-05-01 00:28:26,2022-05-01 00:37:49,1.00,1.60,1.00,238,75,Credit card,7.50,Upper West Side North,East Harlem South,Manhattan,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"


In [215]:
# Ver que tipo de vendor, payment_type, rate_code, store_and_fwd_flag hay
print(df['vendor'].value_counts())
print()
print(df['payment_type'].value_counts())
print()
print(df['rate_code'].value_counts())


vendor
Curb Mobility, LLC                   2480272
Creative Mobile Technologies, LLC    1042980
Name: count, dtype: int64

payment_type
Credit card       2677202
Cash               692441
Flex Fare trip     122295
No charge           15800
Dispute             15514
Name: count, dtype: int64

rate_code
Standard rate            3216781
JFK                       144241
Negocios                   19051
Newark                     10747
Null/unknown                9671
Nassau or Westchester        434
Group ride                    32
Name: count, dtype: int64


In [216]:
# Ver que porcentaje representan los tipos de pago "Dispute"
print(df['payment_type'].value_counts(normalize=True))


payment_type
Credit card      0.76
Cash             0.20
Flex Fare trip   0.03
No charge        0.00
Dispute          0.00
Name: proportion, dtype: float64


Como se puede observar hay tipos de pago "Dispute", los cuales no deberíamos tener en cuenta ya que son viajes que todavía no se cobrado y están en proceso de disputa. Estos se eliminarán, estos repesentan menos del 0.1% de los datos.

In [217]:
# Analizar los valores de payment_type = 4
df = df[df['payment_type'] != "Dispute"]
df['payment_type'].unique()


array(['Cash', 'Credit card', 'No charge', 'Flex Fare trip'], dtype=object)

In [218]:
df.isnull().sum() / len(df) * 100

vendor_id                  0.00
tpep_pickup_datetime       0.00
tpep_dropoff_datetime      0.00
passenger_count            3.49
trip_distance              0.00
ratecode_id                3.49
pu_location_id             0.00
do_location_id             0.00
payment_type               0.00
fare_amount                0.00
pu_location_zone           0.00
do_location_zone           0.00
do_location_district       0.00
pu_location_district       0.00
do_location_service_zone   0.00
pu_location_service_zone   0.00
rate_code                  3.49
vendor                     0.00
dtype: float64

Ahora, veamos los valores negativos en total_amount.

In [219]:
# Verificar valores negativos en fare_amount
df_neg =df[df['fare_amount'] < 0]
df_neg.head()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,payment_type,fare_amount,pu_location_zone,do_location_zone,do_location_district,pu_location_district,do_location_service_zone,pu_location_service_zone,rate_code,vendor
1391,2,2022-05-01 00:27:32,2022-05-01 00:27:37,1.00,0.00,2.00,239,239,No charge,-52.00,Upper West Side South,Upper West Side South,Manhattan,Manhattan,Yellow Zone,Yellow Zone,JFK,"Curb Mobility, LLC"
1575,2,2022-05-01 00:06:25,2022-05-01 00:11:46,1.00,1.21,1.00,162,170,Cash,-6.00,Midtown East,Murray Hill,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Curb Mobility, LLC"
2280,2,2022-05-01 00:10:09,2022-05-01 00:10:57,2.00,0.00,1.00,186,186,No charge,-2.50,Penn Station/Madison Sq West,Penn Station/Madison Sq West,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Curb Mobility, LLC"
2950,2,2022-05-01 00:29:00,2022-05-01 00:29:34,1.00,0.00,1.00,237,237,No charge,-2.50,Upper East Side South,Upper East Side South,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Curb Mobility, LLC"
3229,2,2022-05-01 00:57:55,2022-05-01 00:58:03,1.00,0.03,1.00,259,259,No charge,-2.50,Woodlawn/Wakefield,Woodlawn/Wakefield,Bronx,Bronx,Boro Zone,Boro Zone,Standard rate,"Curb Mobility, LLC"


In [220]:
df_neg.shape[0] / len(df) * 100

0.241409136030114

In [221]:
# Ver que tipo de vendor, payment_type, rate_code, store_and_fwd_flag tienen valores negativos
print(df_neg['vendor'].value_counts())
print()
print(df_neg['payment_type'].value_counts())
print()
print(df_neg['rate_code'].value_counts())


vendor
Curb Mobility, LLC                   8467
Creative Mobile Technologies, LLC       1
Name: count, dtype: int64

payment_type
Cash              4284
No charge         4133
Flex Fare trip      35
Credit card         16
Name: count, dtype: int64

rate_code
Standard rate            7345
JFK                       782
Negocios                  190
Newark                    108
Nassau or Westchester       6
Group ride                  2
Name: count, dtype: int64


In [222]:
df_neg.describe()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,fare_amount
count,"8,468.00",8468,8468,"8,433.00","8,468.00","8,433.00","8,468.00","8,468.00","8,468.00"
mean,2.00,2022-05-16 19:01:45.661903,2022-05-16 19:10:02.938828,1.37,15.54,1.21,160.92,160.15,-13.47
min,1.00,2022-05-01 00:06:25,2022-05-01 00:10:57,0.00,0.00,1.00,1.00,1.00,"-1,311.50"
25%,2.00,2022-05-09 13:43:40,2022-05-09 13:47:08.750000,1.00,0.02,1.00,132.00,130.00,-13.50
50%,2.00,2022-05-16 15:26:54,2022-05-16 15:38:32,1.00,0.58,1.00,161.00,162.00,-6.00
75%,2.00,2022-05-24 13:05:15.500000,2022-05-24 13:10:15,1.00,1.66,1.00,230.00,230.00,-3.00
max,2.00,2022-05-31 23:45:49,2022-05-31 23:47:15,6.00,"116,069.15",6.00,263.00,263.00,-0.01
std,0.01,NaN,NaN,0.84,"1,261.31",0.69,63.91,66.59,22.23


No hemos encontrado una realción a priori de estos valores negativos y, además, estos representan menos del 1% de los datos. Por lo que se procederá a eliminar estos registros.

In [223]:
# Eliminar los registros con valores negativos
df = df[df['fare_amount'] >= 0]
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3499270 entries, 0 to 3588294
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   vendor_id                 3499270 non-null  int64         
 1   tpep_pickup_datetime      3499270 non-null  datetime64[us]
 2   tpep_dropoff_datetime     3499270 non-null  datetime64[us]
 3   passenger_count           3377010 non-null  float64       
 4   trip_distance             3499270 non-null  float64       
 5   ratecode_id               3377010 non-null  float64       
 6   pu_location_id            3499270 non-null  int64         
 7   do_location_id            3499270 non-null  int64         
 8   payment_type              3499270 non-null  object        
 9   fare_amount               3499270 non-null  float64       
 10  pu_location_zone          3499270 non-null  object        
 11  do_location_zone          3499270 non-null  object     

In [224]:
df.isnull().sum() / len(df) * 100

vendor_id                  0.00
tpep_pickup_datetime       0.00
tpep_dropoff_datetime      0.00
passenger_count            3.49
trip_distance              0.00
ratecode_id                3.49
pu_location_id             0.00
do_location_id             0.00
payment_type               0.00
fare_amount                0.00
pu_location_zone           0.00
do_location_zone           0.00
do_location_district       0.00
pu_location_district       0.00
do_location_service_zone   0.00
pu_location_service_zone   0.00
rate_code                  3.49
vendor                     0.00
dtype: float64

Ahora analicemos los valores nulos. Primero veamos si los nulos son independientes.

In [225]:
cols_with_na = df.columns[df.isna().any()]
cols_with_na


Index(['passenger_count', 'ratecode_id', 'rate_code'], dtype='object')

In [226]:
mask_all_na = df[cols_with_na].isna().all(axis=1)
mask_any_na = df[cols_with_na].isna().any(axis=1)

(mask_all_na == mask_any_na).all()

True

Con esto se confirma que los nulos se presentan en las mismas filas. Es decir que todos los nuelos corresponden exactamente a los mismos registros.

In [227]:
df_nulls = df[mask_all_na]
df_nulls.head()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,payment_type,fare_amount,pu_location_zone,do_location_zone,do_location_district,pu_location_district,do_location_service_zone,pu_location_service_zone,rate_code,vendor
3458771,2,2022-05-01 00:01:17,2022-05-01 00:18:38,NaN,4.93,NaN,166,163,Flex Fare trip,23.21,Morningside Heights,Midtown North,Manhattan,Manhattan,Yellow Zone,Boro Zone,NaN,"Curb Mobility, LLC"
3458772,2,2022-05-01 00:40:00,2022-05-01 00:50:00,NaN,1.48,NaN,190,97,Flex Fare trip,10.00,Prospect Park,Fort Greene,Brooklyn,Brooklyn,Boro Zone,Boro Zone,NaN,"Curb Mobility, LLC"
3458773,1,2022-05-01 00:08:38,2022-05-01 00:28:26,NaN,3.90,NaN,239,234,Flex Fare trip,16.50,Upper West Side South,Union Sq,Manhattan,Manhattan,Yellow Zone,Yellow Zone,NaN,"Creative Mobile Technologies, LLC"
3458774,2,2022-05-01 00:34:59,2022-05-01 01:12:00,NaN,13.16,NaN,61,151,Flex Fare trip,43.79,Crown Heights North,Manhattan Valley,Manhattan,Brooklyn,Yellow Zone,Boro Zone,NaN,"Curb Mobility, LLC"
3458775,2,2022-05-01 00:12:00,2022-05-01 00:25:00,NaN,4.29,NaN,79,262,Flex Fare trip,17.65,East Village,Yorkville East,Manhattan,Manhattan,Yellow Zone,Yellow Zone,NaN,"Curb Mobility, LLC"


In [228]:
def missingness_summary(df, category_col, mask_na):
    summary = pd.DataFrame({
        'total_rows': df.groupby(category_col).size(),
        'rows_with_na': df[mask_na].groupby(category_col).size()
    }).fillna(0)

    summary['pct_of_total_df'] = summary['total_rows'] / len(df) * 100
    summary['pct_rows_with_na'] = summary['rows_with_na'] / summary['total_rows'] * 100

    return summary.sort_values('pct_of_total_df', ascending=False)


In [229]:
missingness_summary(df, 'vendor', mask_all_na)


,total_rows,rows_with_na,pct_of_total_df,pct_rows_with_na
vendor,,,,
"Curb Mobility, LLC",2460517,108838,70.32,4.42
"Creative Mobile Technologies, LLC",1038753,13422,29.68,1.29


In [230]:
missingness_summary(df, 'payment_type', mask_all_na)

,total_rows,rows_with_na,pct_of_total_df,pct_rows_with_na
payment_type,,,,
Credit card,2677186,0.00,76.51,0.00
Cash,688157,0.00,19.67,0.00
Flex Fare trip,122260,"122,260.00",3.49,100.00
No charge,11667,0.00,0.33,0.00


In [231]:
missingness_summary(df, 'rate_code', mask_all_na)

,total_rows,rows_with_na,pct_of_total_df,pct_rows_with_na
rate_code,,,,
Standard rate,3195475,0.00,91.32,0.00
JFK,142558,0.00,4.07,0.00
Negocios,18433,0.00,0.53,0.00
Newark,10435,0.00,0.30,0.00
Null/unknown,9671,0.00,0.28,0.00
Nassau or Westchester,410,0.00,0.01,0.00
Group ride,28,0.00,0.00,0.00


Como se puede observar, los nulos son todos los viajes con tipo de pago "Flex Fare trip". Procedemos a eliminar todos los nulos ya que no representan un porcentaje significativo de los datos.

In [232]:
# Eliminar las filas con nulos
df = df.dropna()

# Verificar resultados
df.isnull().sum().sum()

0

In [233]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3377010 entries, 0 to 3458770
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   vendor_id                 3377010 non-null  int64         
 1   tpep_pickup_datetime      3377010 non-null  datetime64[us]
 2   tpep_dropoff_datetime     3377010 non-null  datetime64[us]
 3   passenger_count           3377010 non-null  float64       
 4   trip_distance             3377010 non-null  float64       
 5   ratecode_id               3377010 non-null  float64       
 6   pu_location_id            3377010 non-null  int64         
 7   do_location_id            3377010 non-null  int64         
 8   payment_type              3377010 non-null  object        
 9   fare_amount               3377010 non-null  float64       
 10  pu_location_zone          3377010 non-null  object        
 11  do_location_zone          3377010 non-null  object     

In [234]:
# Ver que porcentaje de datos se eliminaron en total
(rows_before - len(df)) / rows_before * 100

5.888172516473701

In [235]:
# # Eliminar las columnas con id
# df = df.drop(columns=['pu_location_id', 'do_location_id', 'ratecode_id', 'vendor_id'])

# # Verificar resultados
# df.head()

Asimismo, vemos que hay viajes con distancia 0, veamos estos casos

In [236]:
df_dist_0 = df[df['trip_distance'] == 0]
df_dist_0.shape[0] / len(df) * 100


1.004586897877116

In [237]:
df_dist_0.describe()



,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,fare_amount
count,"33,925.00",33925,33925,"33,925.00","33,925.00","33,925.00","33,925.00","33,925.00","33,925.00"
mean,1.60,2022-05-16 16:22:55.912601,2022-05-16 16:34:33.873898,1.31,0.00,7.63,148.77,147.55,26.17
min,1.00,2022-05-01 00:01:59,2022-05-01 00:02:29,0.00,0.00,1.00,1.00,1.00,0.00
25%,1.00,2022-05-09 12:00:31,2022-05-09 12:05:42,1.00,0.00,1.00,112.00,107.00,2.50
50%,2.00,2022-05-16 17:53:58,2022-05-16 18:03:41,1.00,0.00,1.00,145.00,145.00,15.00
75%,2.00,2022-05-24 00:21:56,2022-05-24 00:31:14,1.00,0.00,5.00,202.00,194.00,52.00
max,2.00,2022-05-31 23:58:06,2022-06-01 17:17:10,9.00,0.00,99.00,263.00,263.00,900.00
std,0.49,NaN,NaN,0.88,0.00,22.19,67.48,67.95,29.11


In [238]:
print(df['rate_code'].value_counts(normalize=True))
print()
print(df['payment_type'].value_counts(normalize=True))
print()
print(df['vendor'].value_counts(normalize=True))



rate_code
Standard rate           0.95
JFK                     0.04
Negocios                0.01
Newark                  0.00
Null/unknown            0.00
Nassau or Westchester   0.00
Group ride              0.00
Name: proportion, dtype: float64

payment_type
Credit card   0.79
Cash          0.20
No charge     0.00
Name: proportion, dtype: float64

vendor
Curb Mobility, LLC                  0.70
Creative Mobile Technologies, LLC   0.30
Name: proportion, dtype: float64


We know that duration and fare are frequently high because the upper quartile (p75) already shows large values, indicating that a substantial fraction of zero-distance trips have long durations and high fares, not just isolated outliers.

En base a esto, eliminaremos los viajes con distancia 0.

In [239]:
df = df[df['trip_distance'] != 0]
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3343085 entries, 0 to 3458770
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   vendor_id                 3343085 non-null  int64         
 1   tpep_pickup_datetime      3343085 non-null  datetime64[us]
 2   tpep_dropoff_datetime     3343085 non-null  datetime64[us]
 3   passenger_count           3343085 non-null  float64       
 4   trip_distance             3343085 non-null  float64       
 5   ratecode_id               3343085 non-null  float64       
 6   pu_location_id            3343085 non-null  int64         
 7   do_location_id            3343085 non-null  int64         
 8   payment_type              3343085 non-null  object        
 9   fare_amount               3343085 non-null  float64       
 10  pu_location_zone          3343085 non-null  object        
 11  do_location_zone          3343085 non-null  object     

In [240]:
df.describe()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,fare_amount
count,"3,343,085.00",3343085,3343085,"3,343,085.00","3,343,085.00","3,343,085.00","3,343,085.00","3,343,085.00","3,343,085.00"
mean,1.70,2022-05-16 08:30:08.360514,2022-05-16 08:48:13.218197,1.40,3.58,1.29,163.76,162.07,14.57
min,1.00,2003-01-01 00:06:06,2003-01-01 00:31:38,0.00,0.01,1.00,1.00,1.00,0.00
25%,1.00,2022-05-08 19:08:51,2022-05-08 19:25:53,1.00,1.16,1.00,132.00,113.00,7.00
50%,2.00,2022-05-16 10:14:54,2022-05-16 10:32:12,1.00,1.92,1.00,161.00,162.00,10.50
75%,2.00,2022-05-23 18:38:18,2022-05-23 18:54:41,1.00,3.60,1.00,233.00,234.00,16.00
max,2.00,2022-06-01 23:55:30,2022-06-02 00:03:51,8.00,"29,445.65",99.00,263.00,263.00,"6,966.50"
std,0.46,NaN,NaN,0.96,17.97,4.73,64.64,69.75,12.98


Por último, veremos si hay viajes donde el dropoff time sean menor al pickup time, esto no es posible.

In [246]:
df_dropoff_before_pickup = df[df['tpep_dropoff_datetime'] <= df['tpep_pickup_datetime']]
df_dropoff_before_pickup.head()


,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,payment_type,fare_amount,pu_location_zone,do_location_zone,do_location_district,pu_location_district,do_location_service_zone,pu_location_service_zone,rate_code,vendor
5707,1,2022-05-01 01:12:42,2022-05-01 01:12:42,2.00,3.10,1.00,186,186,Cash,12.00,Penn Station/Madison Sq West,Penn Station/Madison Sq West,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
75246,1,2022-05-01 19:51:12,2022-05-01 19:51:12,2.00,1.00,1.00,262,262,Cash,6.50,Yorkville East,Yorkville East,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
99666,1,2022-05-02 06:39:07,2022-05-02 06:39:07,1.00,1.20,1.00,186,186,Cash,6.50,Penn Station/Madison Sq West,Penn Station/Madison Sq West,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
131520,1,2022-05-02 13:13:28,2022-05-02 13:13:28,1.00,0.80,1.00,170,170,Cash,5.50,Murray Hill,Murray Hill,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"
219392,1,2022-05-03 12:13:26,2022-05-03 12:13:26,1.00,2.60,1.00,107,224,No charge,14.00,Gramercy,Stuy Town/Peter Cooper Village,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC"


In [247]:
print(len(df_dropoff_before_pickup))
print(len(df_dropoff_before_pickup) / len(df) * 100)

80
0.002392999280604591


Vemos que tenemos 80 viajes donde el dropoff time es menor o igual  al pickup time, estos se eliminarán también.

In [248]:
df = df[df['tpep_dropoff_datetime'] > df['tpep_pickup_datetime']]
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 3343005 entries, 0 to 3458770
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   vendor_id                 3343005 non-null  int64         
 1   tpep_pickup_datetime      3343005 non-null  datetime64[us]
 2   tpep_dropoff_datetime     3343005 non-null  datetime64[us]
 3   passenger_count           3343005 non-null  float64       
 4   trip_distance             3343005 non-null  float64       
 5   ratecode_id               3343005 non-null  float64       
 6   pu_location_id            3343005 non-null  int64         
 7   do_location_id            3343005 non-null  int64         
 8   payment_type              3343005 non-null  object        
 9   fare_amount               3343005 non-null  float64       
 10  pu_location_zone          3343005 non-null  object        
 11  do_location_zone          3343005 non-null  object     

In [249]:
df.describe()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,fare_amount
count,"3,343,005.00",3343005,3343005,"3,343,005.00","3,343,005.00","3,343,005.00","3,343,005.00","3,343,005.00","3,343,005.00"
mean,1.70,2022-05-16 08:30:02.628596,2022-05-16 08:48:07.512495,1.40,3.58,1.29,163.76,162.07,14.57
min,1.00,2003-01-01 00:06:06,2003-01-01 00:31:38,0.00,0.01,1.00,1.00,1.00,0.00
25%,1.00,2022-05-08 19:08:46,2022-05-08 19:25:48,1.00,1.16,1.00,132.00,113.00,7.00
50%,2.00,2022-05-16 10:14:44,2022-05-16 10:31:57,1.00,1.92,1.00,161.00,162.00,10.50
75%,2.00,2022-05-23 18:38:12,2022-05-23 18:54:38,1.00,3.60,1.00,233.00,234.00,16.00
max,2.00,2022-06-01 23:55:30,2022-06-02 00:03:51,8.00,"29,445.65",99.00,263.00,263.00,"6,966.50"
std,0.46,NaN,NaN,0.96,17.97,4.73,64.64,69.75,12.98


In [250]:
# Ver que porcentaje de datos se eliminaron en total
(rows_before - len(df)) / rows_before * 100

6.835837075825705

Ahora que tenemos las columnas con las categorías, procedemremos a hacer un análisis más profundo de las variables.

## Análisis de datos

### Análisis de variables numéricas

In [93]:
# Determinando las columnas de tipo integer y float

numeric_columns = [
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'trip_duration'
    ]

categorical_columns = [
    'vendor',
    'rate_code',
    'payment_type',
    'pu_location_zone',
    'pu_location_district',
    'pu_location_service_zone',
    'do_location_zone',
    'do_location_district',
    'do_location_service_zone'
    ]

In [94]:
df.head()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,payment_type,fare_amount,pu_location_zone,do_location_zone,do_location_district,pu_location_district,do_location_service_zone,pu_location_service_zone,rate_code,vendor,trip_duration
0,1,2022-05-01 00:00:36,2022-05-01 00:19:18,1.00,4.10,1.00,246,151,Cash,17.00,West Chelsea/Hudson Yards,Manhattan Valley,Manhattan,Manhattan,Yellow Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC",18.70
1,1,2022-05-01 00:27:44,2022-05-01 00:41:33,1.00,2.30,1.00,238,74,Cash,11.00,Upper West Side North,East Harlem North,Manhattan,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC",13.82
2,1,2022-05-01 00:59:00,2022-05-01 01:14:22,1.00,4.20,1.00,163,260,Cash,15.50,Midtown North,Woodside,Queens,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC",15.37
3,1,2022-05-01 00:48:18,2022-05-01 01:28:02,1.00,0.00,1.00,79,182,Credit card,41.20,East Village,Parkchester,Bronx,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC",39.73
4,1,2022-05-01 00:28:26,2022-05-01 00:37:49,1.00,1.60,1.00,238,75,Credit card,7.50,Upper West Side North,East Harlem South,Manhattan,Manhattan,Boro Zone,Yellow Zone,Standard rate,"Creative Mobile Technologies, LLC",9.38


In [95]:
df.describe()

,vendor_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecode_id,pu_location_id,do_location_id,fare_amount,trip_duration
count,"3,377,010.00",3377010,3377010,"3,377,010.00","3,377,010.00","3,377,010.00","3,377,010.00","3,377,010.00","3,377,010.00","3,377,010.00"
mean,1.70,2022-05-16 08:34:53.337225,2022-05-16 08:52:54.308200,1.39,3.54,1.35,163.61,161.92,14.69,18.02
min,1.00,2003-01-01 00:06:06,2003-01-01 00:31:38,0.00,0.00,1.00,1.00,1.00,0.00,-14.08
25%,1.00,2022-05-08 19:14:40,2022-05-08 19:31:27,1.00,1.13,1.00,132.00,113.00,7.00,7.60
50%,2.00,2022-05-16 10:21:06.500000,2022-05-16 10:38:26.500000,1.00,1.90,1.00,161.00,162.00,10.50,12.50
75%,2.00,2022-05-23 18:40:14.750000,2022-05-23 18:56:20.750000,1.00,3.57,1.00,233.00,234.00,16.50,20.20
max,2.00,2022-06-01 23:55:30,2022-06-02 00:03:51,9.00,"29,445.65",99.00,263.00,263.00,"6,966.50","6,823.55"
std,0.46,NaN,NaN,0.96,17.89,5.25,64.69,69.75,13.29,52.46


In [ ]:
sns.set(style="ticks")

num_vars = len(numeric_columns)
num_rows = num_vars // 2  # assuming we want 2 plots per row
num_rows += num_vars % 2  # add an extra row if there are leftovers

# create the subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(15, num_rows*5))  # Change the size as you see fit.

# flatten the axes array, so we can easily iterate over it
axes = axes.flatten()

# iterate over your numeric columns and plot a histogram on the corresponding axes
for i, col in enumerate(numeric_columns):
    axes[i].hist(df[col], bins=16, color='slategray', edgecolor="black", linewidth=0.8)
    
    # Draw vertical lines for mean and median
    mean_val = df[col].mean()
    median_val = df[col].median()
    mean_line = axes[i].axvline(mean_val, color='tomato', linestyle='dashed', linewidth=2)
    median_line = axes[i].axvline(median_val, color='dodgerblue', linestyle='dashed', linewidth=2)
    
    # Add legend
    axes[i].legend([mean_line, median_line], ['mean', 'median'])
    axes[i].set_title(f'Histogram of {col}', fontsize=16)
    
# remove the unused subplots
if len(numeric_columns) % 2:
    for j in range(i+1, num_rows*2):
        fig.delaxes(axes[j])
    
plt.tight_layout()
plt.show()

In [ ]:
sns.set(style="ticks")

num_vars = len(numeric_columns)
num_rows = num_vars // 2  # assuming we want 2 plots per row
num_rows += num_vars % 2  # add an extra row if there are leftovers

# create the subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(15, num_rows*5))

# flatten the axes array, so we can easily iterate over it
axes = axes.flatten()

# iterate over your numeric columns and plot a box plot on the corresponding axes
for i, col in enumerate(numeric_columns):
    sns.boxplot(x=df[col], color='slategray', ax=axes[i])
    axes[i].set_title(f'Boxplot of {col}', fontsize=20, weight='bold')
    axes[i].set_xlabel(col, fontsize=16)
    
    # Draw vertical lines for mean and median
    mean_val = df[col].mean()
    median_val = df[col].median()
    mean_line = axes[i].axvline(mean_val, color='tomato', linestyle='dashed', linewidth=2)
    median_line = axes[i].axvline(median_val, color='dodgerblue', linestyle='dashed', linewidth=2)

    # Add legend
    axes[i].legend([mean_line, median_line], ['mean', 'median'])
    axes[i].set_title(f'Boxplot of {col}', fontsize=16)
    
plt.tight_layout()
plt.show()

Viendo los boxplots e histogramas, podemos observar que los datos en varias varibales no son simétricos, es decir hay muchos outliers. Los casos más extremos se ven en la distancia de los viajes, donde hay viajes de más de 100 millas y el tiempo de viaje, donde hay viajes de más de 3 horas. Ahora veamos la correlación entre las variables numéricas.

In [ ]:
# Calculate the correlation matrix
import numpy as np
corr = df[numeric_columns].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
plt.figure(figsize=(15, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0, annot=True, fmt=".3f", annot_kws={'size': 10},
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

# Add a title to the heatmap
plt.title('Correlation Heatmap', fontsize=20, fontweight='bold')

# Display the heatmap
plt.show()

Como se suponía los costos de los viajes son las variables más relacionadas entre si. Por otro lado, no se muestra una relación fuerte entre el tiempo de viaje y el costo del viaje.

### Anáisis de variables categóricas

In [ ]:
# set the seaborn style
sns.set(style="ticks")

# determine the number of rows needed for our subplots
num_vars = len(categorical_columns)
num_rows = num_vars // 2
num_rows += num_vars % 2

# create the subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(20, num_rows*10))
axes = axes.flatten()

total_records = len(df)

for i, column in enumerate(categorical_columns):
    column_counts = df[column].value_counts().nlargest(15)
    column_percentages = (column_counts / total_records * 100)
    
    # Crear gráfico de barras con porcentajes
    bars = column_percentages.plot(kind='bar', ax=axes[i], color="slategray", edgecolor='black')
    
    axes[i].set_title(f'Barplot for {column}', fontsize=22)
    axes[i].set_xlabel(column, fontsize=16)
    axes[i].set_ylabel('Percentage (%)', fontsize=16)
    axes[i].tick_params(axis='x', rotation=75, labelsize=15)
    axes[i].tick_params(axis='y', labelsize=15)
    
    # Añadir los valores de porcentaje sobre cada barra
    for container in axes[i].containers:
        axes[i].bar_label(container, fmt='%.2f%%', fontsize=12, padding=3)

plt.tight_layout()
plt.show()

Respecto a al columnas categóricas, este es lo que se puede observar:
- **vendor**: El vendor más común es Creative Mobile Technologies, LLC con alrededor de 70% de los viajes.
- **rate_code**: La tarifa más común es la estándar (70% de los viajes aprox), la segunda más común es la de JFK (5% de los viajes aprox).
- **payment_type**: El tipo de pago más común es es el credit card con alrededor de 80% de los viajes y la segunda más común es el cash con alrededor de 15% de los viajes.
- **pu_location_district**: Manhattan concentra más del 85% de los pickups.
- **pu_location_service_zone**: Yellow Zone representa más del 85% de los pickups y la segunda más común son loas aereopuertos (~7%).
- **do_location_district**: Manhattan concentra más del 85% de los dropoffs
- **do_location_service_zone**: Patrón similar a pickups, Yellow Zone domina con ~80% y Boro zone es la siguiente con ~12%.
- **store_and_fwd_flag**: la categoría "N" (No) representa más del 95% de los viajes.


Ahora mostraremos un gráfico de pares  para mostrar la relación entre las variables.

In [ ]:
# Graficar gráfico de pares
g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

### Conclusiones y siguientes pasos

## Procesamiento de datos

### Tratamiento de valores ausentes

Lo primero que derteminaremos es si estos valores ausentes son independientes o no.

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
# Cantidad de nulos
df.isnull().sum() / len(df) * 100

In [ ]:
# Columnas con nulos
cols_con_nulos = df.columns[df.isna().any()].tolist()

# Verificación final: ¿Cuando una es nula, TODAS son nulas?
print("\n Verificación: ¿Cuando falta una, faltan todas?")
print("-" * 70)

# Tomar la primera columna como referencia
col_referencia = cols_con_nulos[0]
registros_nulos_ref = df[col_referencia].isna()

# Verificar si coincide con las demás
todas_coinciden = True
for col in cols_con_nulos[1:]:
    coinciden = (df[col].isna() == registros_nulos_ref).all()
    simbolo = "✅" if coinciden else "❌"
    print(f"{simbolo} {col_referencia} vs {col}: {'Coinciden 100%' if coinciden else 'NO coinciden'}")
    if not coinciden:
        todas_coinciden = False


Al parecer, los nulos son independientes. Es decir los nulos coinciden en las mismas filas. Ahora procederemos a buscar la causa común de los nulos, para ello compararemos los valores de las columnas categóricas de las filas con nulos con las mismas columnas de las filas sin nulos.

In [ ]:
# Crear máscara de registros con nulos (usar cualquier columna, son las mismas)
tiene_nulos = df['passenger_count'].isna()


# Campos categóricos clave a investigar
campos_investigar = ['vendor', 'payment_type', 'rate_code', 'store_and_fwd_flag']

for campo in campos_investigar:
    print(f"\n🔍 Campo: {campo}")
    print("-" * 70)
    
    # Ver valores únicos en registros CON nulos
    valores_en_nulos = df[tiene_nulos][campo].value_counts(dropna=False)
    
    # Comparar
    print(f"Valores en registros CON nulos:")
    print(valores_en_nulos)
    
    # TEST CLAVE: ¿Algún valor tiene 100% de nulos?
    print(f"\n% de nulos por cada valor de {campo}:")
    for valor in df[campo].dropna().unique():
        registros_con_este_valor = df[campo] == valor
        nulos_en_este_valor = (df[registros_con_este_valor]['passenger_count'].isna()).sum()
        total_este_valor = registros_con_este_valor.sum()
        pct = (nulos_en_este_valor / total_este_valor * 100) if total_este_valor > 0 else 0
        
        if pct == 100.0:
            print(f"   {valor}: {pct:.1f}% ⚠️ ¡TODOS los registros tienen nulos!")
        elif pct > 50:
            print(f"   {valor}: {pct:.1f}% ⚠️")
        elif pct > 0:
            print(f"   {valor}: {pct:.1f}%")
        else:
            print(f"   {valor}: {pct:.1f}%")


Hemos encontrado que todos los registros nulos son del tipo de pago Flex Fare trip (0), este tipo de pago es relacionado a taxis por aplicativo. Por otro lado, tenemos dos vendors que tienen un 100% de nulos, Creative Mobile Technologies, LLC y Curb Mobility, LLC (relacionado con apps de taxis).

In [ ]:
# Filtrar registros con payment_type = 0
df_flex = df[df['payment_type'] == 0]

df_flex.describe()

In [ ]:
# Filtrar payment_type = 0
df = df[df['payment_type'] != "Flex Fare trip"]

df['payment_type'].value_counts()

### Tratamiento de outliers

##### Trip Distance

Pasemos a estudiar la columna trip_distance.

In [ ]:
# Verificar datos estadísticos de la columna trip_distance
df['trip_distance'].describe()


Como podemos observar, el boxplot muestra que hay varios viajes con distancias atípicas. Esto puede ser porque hay viajes que no son dentro de New York, por lo que se procederá a eliminar estos viajes. Veamos el contenido completo de los valores con la frecuencia de ocurrencia.

In [ ]:
# Value counts con distancias redondeadas (sin decimales)
distancias_redondeadas = df['trip_distance'].round(0).astype(int)
value_counts = distancias_redondeadas.value_counts().sort_index(ascending=False)

print(f"{'Distancia (mi)':>15}  {'Cantidad':>10}")
print("="*30)

for distancia, cantidad in value_counts.items():
    print(f"{distancia:>15}  {cantidad:>10,}")

In [ ]:
sns.set(style="ticks")

# Definir las "columnas numéricas" a graficar (específico para estos dos subplots)
hist_columns = [
    'trip_distance',  # Histograma completo
    'trip_distance_zoom',  # Histograma para <= 50 millas (agregada como serie temporal)
]

# Crear una columna temporal para el zoom
df['trip_distance_zoom'] = df['trip_distance'].where(df['trip_distance'] <= 50)

num_vars = len(hist_columns)
num_rows = num_vars // 2
num_rows += num_vars % 2

fig, axes = plt.subplots(num_rows, 2, figsize=(16, num_rows * 7))
axes = axes.flatten()

for i, col in enumerate(hist_columns):
    # Seleccionar la serie (filtrado para la columna de zoom)
    if col == 'trip_distance_zoom':
        data = df['trip_distance_zoom'].dropna()
        hist_title = 'Distribución (≤50 millas)'
    else:
        data = df['trip_distance']
        hist_title = 'Distribución Completa'
    
    # Escala log solo en el histograma completo
    if col == 'trip_distance':
        axes[i].hist(data, bins=100, color='slategray', edgecolor="black", linewidth=0.8)
        axes[i].set_yscale('log')
    else:
        axes[i].hist(data, bins=100, color='slategray', edgecolor="black", linewidth=0.8)
    
    axes[i].set_title(hist_title)
    axes[i].set_xlabel('Distancia (millas)')
    axes[i].set_ylabel('Frecuencia')
    
    mean_val = data.mean()
    median_val = data.median()
    mean_line = axes[i].axvline(mean_val, color='tomato', linestyle='dashed', linewidth=2)
    median_line = axes[i].axvline(median_val, color='dodgerblue', linestyle='dashed', linewidth=2)
    
    print(f'{hist_title}:')
    print(f'Median: {median_val}')
    print(f'Mean: {mean_val}')
    print()
    axes[i].legend([mean_line, median_line], ['mean', 'median'])
    
# Remover subgráficos no usados
for j in range(i+1, num_rows*2):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

# Eliminar la columna auxiliar para no dejar basura en el DataFrame
df.drop(columns=['trip_distance_zoom'], inplace=True, errors='ignore')

Como se puede observar, hay una cola larga a la derecha, lo cual indica que hay una asimetría en la distribución, la concentración de datos se encuentra en las distancias cortas. Además, se puede observar que hay un salto en la distancia de 50 millas, lo cual sugiere que hay un umbral para la distancia de los viajes. Comprobemos esto midiendo el skewness y el kurtosis.

In [ ]:
skewness = scipy_stats.skew(df['trip_distance'])
kurtosis = scipy_stats.kurtosis(df['trip_distance'])

print(f"  Skewness (asimetría):  {skewness:.2f}")
print(f"  Kurtosis (curtosis):   {kurtosis:.2f}")

El skewness indica que la distribución es asimétrica a la derecha (skewness > 0), lo cual es consistente con la observación de la cola larga en el histograma. El kurtosis es muy alto, lo cual indica que la distribución es muy puntiaguda (concentrada) y tiene muchos outliers.

Sabiendo esto sabemos que no podemos usar métodos que asumen normalidad, como el Z-score, ya que la distribución no es normal. Por otro lado, los métodos robustos como el MAD y los percentiles son más adecuados para detectar outliers.

Validaremos los umbrales de los métodos robustos para detectar outliers segun la distrubución atípica de la distancia de los viajes. Usaremos los métodos IQR, MAD y percentiles para validar los umbrales.

In [ ]:

# Método 1: IQR
Q1 = df['trip_distance'].quantile(0.25)
Q3 = df['trip_distance'].quantile(0.75)
IQR = Q3 - Q1
umbral_iqr = Q3 + 1.5 * IQR

# Método 2: MAD
mediana = df['trip_distance'].median()
mad = scipy_stats.median_abs_deviation(df['trip_distance'])
umbral_mad = mediana + (3.5 * mad / 0.6745)

# Método 3: Percentiles
umbrales_percentiles = {
    '99.0%': df['trip_distance'].quantile(0.990),
    '99.5%': df['trip_distance'].quantile(0.995),
    '99.9%': df['trip_distance'].quantile(0.999),
}

# Tabla comparativa
print(f"\n{'Método':<20} {'Umbral':>12} {'N Outliers':>12} {'% Outliers':>12}")
print("-"*60)

metodos = {
    'IQR (Q3+1.5*IQR)': umbral_iqr,
    'MAD (Med+3.5*MAD)': umbral_mad,
    **umbrales_percentiles
}

for nombre, umbral in metodos.items():
    n_outliers = (df['trip_distance'] > umbral).sum()
    pct = (n_outliers / len(df)) * 100
    print(f"{nombre:<20} {umbral:>10.2f} mi {n_outliers:>10,}   {pct:>10.2f}%")

Como se puedo observar el método menos agresivo es usanod percentiles y va de la mano con la lógica con la distacia máxima que podria tener un viaje en la ciudad de Nueva York. Hagamos un análisis más profundo con los percentiles para establcer el más adecuado.

In [ ]:
# Probar rangos cercanos
percentiles_test = [99.0, 99.5, 99.8, 99.9, 99.95, 99.99]

print(f"\n{'Percentil':<12} {'Umbral':>10} {'Elimina':>10} {'%':>8}")
print("-"*45)

for p in percentiles_test:
    umbral = df['trip_distance'].quantile(p/100)
    n_elimina = (df['trip_distance'] > umbral).sum()
    pct = (n_elimina / len(df)) * 100
    print(f"{p:>6.2f}%    {umbral:>8.2f} mi {n_elimina:>8,}   {pct:>6.3f}%")


El más adecuado es el percentil 99.95%, ya que elimina alrededor de 0.05% de los datos (1,734 registros), lo cual es muy poco y abarcar los viajes con distancias reales dentro de New York. Por lo tanto, usaremos este umbral para eliminar los outliers.

In [ ]:
# Filtrar outliers
umbral = df['trip_distance'].quantile(0.9995)
df = df[df['trip_distance'] <= umbral]

# Verificar resultados
print(f"\nTotal de viajes después de eliminar outliers: {len(df):,}")

In [ ]:
df['trip_distance'].describe()

In [ ]:
# Graficar histograma de una columna específica
sns.set(style="ticks")

# Crear figura
fig, ax = plt.subplots(figsize=(10, 6))

# Limite
limit = 10

# Crear histograma
ax.hist(df['trip_distance'], bins=100, color='slategray', edgecolor="black", linewidth=0.8)

# Dibujar líneas para media y mediana
mean_val = df['trip_distance'].mean()
median_val = df['trip_distance'].median()
mean_line = ax.axvline(mean_val, color='tomato', linestyle='dashed', linewidth=2)
median_line = ax.axvline(median_val, color='dodgerblue', linestyle='dashed', linewidth=2)

# Imprimir estadísticas
print(f'trip_distance:')
print(f'Median: {median_val}')
print(f'Mean: {mean_val}')

# Añadir leyenda
ax.legend([mean_line, median_line], ['mean', 'median'])
ax.set_title(f'Histogram of trip_distance', fontsize=16)
ax.set_xlabel('trip_distance')
ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplot de trip_distance

plt.figure(figsize=(10, 6))
sns.boxplot(y=df['trip_distance'])
plt.title('Boxplot of trip_distance')
plt.ylabel('Trip Distance')
plt.show()

In [ ]:
# Value count ordenado por distancia (ordenar por valor/distancia, no por frecuencia)
df['trip_distance'].value_counts().sort_index(ascending=False)


In [ ]:
zero_distance_trips = df[df['trip_distance'] == 0]
zero_distance_trips.head()

In [ ]:
# Porcentaje de viajes con distancia 0
zero_distance_trips.shape[0] / len(df) * 100

In [ ]:
sns.set(style="ticks")

num_vars = len(numeric_columns)
num_rows = num_vars // 2  # assuming we want 2 plots per row
num_rows += num_vars % 2  # add an extra row if there are leftovers

# create the subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(15, num_rows*5))  # Change the size as you see fit.

# flatten the axes array, so we can easily iterate over it
axes = axes.flatten()

# iterate over your numeric columns and plot a histogram on the corresponding axes
for i, col in enumerate(numeric_columns):
    axes[i].hist(zero_distance_trips[col], bins=16, color='slategray', edgecolor="black", linewidth=0.8)
    
    # Draw vertical lines for mean and median
    mean_val = zero_distance_trips[col].mean()
    median_val = zero_distance_trips[col].median()
    mean_line = axes[i].axvline(mean_val, color='tomato', linestyle='dashed', linewidth=2)
    median_line = axes[i].axvline(median_val, color='dodgerblue', linestyle='dashed', linewidth=2)
    
    # Add legend
    axes[i].legend([mean_line, median_line], ['mean', 'median'])
    axes[i].set_title(f'Histogram of {col}', fontsize=16)
    
# remove the unused subplots
if len(numeric_columns) % 2:
    for j in range(i+1, num_rows*2):
        fig.delaxes(axes[j])
    
plt.tight_layout()
plt.show()

Ya que los viajes con distacia zero representan solo el 1% aproximadamente, los eliminaremos.

In [ ]:
df = df[df['trip_distance'] > 0]
df.info(show_counts=True)

In [ ]:
# Ver distancias únicas ordenadas de mayor a menor
distancias_ordenadas = df['trip_distance'].sort_values(ascending=False).drop_duplicates().reset_index(drop=True)

# Mostrar las primeras 100 para ver dónde está el salto
print("Top 100 distancias únicas (de mayor a menor):")
print("="*60)
print(f"{'#':<5} {'Distancia':<15} {'Frecuencia':<12}")
print("-"*60)

for i in range(min(200, len(distancias_ordenadas))):
    dist = distancias_ordenadas[i]
    freq = (df['trip_distance'] == dist).sum()
    print(f"{i+1:<5} {dist:<15.2f} {freq:<12}")

### Tratamiento de outliers

Ahora procederemos a estudiar los outliers de las columnas trip_distance, fare_amount, total_amount y tip_amount.